In [1]:
import random
import pandas as pd
import numpy as np

# Generate Sensors

In [4]:
grid_len = 64
all_locations = []
for i in range(grid_len):
    for j in range(grid_len):
        all_locations.append((i, j))

num_sensor = 1000
sensors = random.sample(all_locations, num_sensor)
sensors

[(28, 48),
 (60, 26),
 (15, 45),
 (24, 46),
 (43, 10),
 (4, 10),
 (50, 22),
 (12, 50),
 (14, 16),
 (53, 19),
 (47, 31),
 (24, 8),
 (33, 4),
 (44, 3),
 (48, 12),
 (62, 36),
 (26, 26),
 (23, 17),
 (24, 21),
 (40, 45),
 (33, 58),
 (48, 45),
 (43, 38),
 (51, 25),
 (59, 51),
 (36, 2),
 (1, 18),
 (53, 51),
 (17, 57),
 (12, 43),
 (18, 23),
 (51, 52),
 (8, 63),
 (28, 17),
 (38, 3),
 (10, 39),
 (4, 49),
 (48, 48),
 (55, 20),
 (38, 51),
 (63, 8),
 (2, 35),
 (0, 16),
 (6, 39),
 (9, 39),
 (22, 40),
 (15, 53),
 (48, 0),
 (59, 3),
 (5, 44),
 (62, 61),
 (30, 41),
 (56, 26),
 (54, 30),
 (26, 44),
 (34, 34),
 (41, 37),
 (21, 30),
 (46, 40),
 (45, 30),
 (34, 23),
 (20, 1),
 (43, 43),
 (8, 53),
 (54, 31),
 (32, 24),
 (19, 12),
 (44, 51),
 (59, 24),
 (9, 14),
 (16, 43),
 (12, 25),
 (53, 31),
 (58, 7),
 (10, 30),
 (31, 12),
 (32, 6),
 (49, 6),
 (44, 8),
 (41, 53),
 (57, 3),
 (3, 37),
 (46, 61),
 (31, 25),
 (38, 1),
 (45, 26),
 (61, 25),
 (56, 17),
 (56, 1),
 (8, 10),
 (31, 62),
 (15, 30),
 (3, 40),
 (12, 3

In [5]:
ipsn = pd.read_csv('sensor_from_ipsn', delimiter=' ', header=None)
stds = ipsn[2]
all_stds = np.unique(stds.values)
all_stds

array([0.31961585, 0.41428093, 0.41543918, 0.42026762, 0.45186382,
       0.51815505, 0.6606852 , 0.7535492 , 1.04563019, 1.12004327,
       1.43466257, 1.52048246, 1.83061047, 1.85433775, 1.86264196,
       2.02253412, 2.34168195, 2.57483698])

In [6]:
std_costs = []
for sen in sensors:
    std_costs.append((random.choice(all_stds), 1))
    
for std_cost in std_costs:
    print(std_cost)

(0.4202676150502109, 1)
(0.31961585322723524, 1)
(0.4142809311761813, 1)
(1.8543377546204012, 1)
(1.4346625681183898, 1)
(0.4518638216531902, 1)
(0.5181550542899774, 1)
(0.4518638216531902, 1)
(1.8543377546204012, 1)
(1.4346625681183898, 1)
(2.341681953932289, 1)
(0.7535492018748609, 1)
(0.4518638216531902, 1)
(2.02253412315468, 1)
(1.5204824599919968, 1)
(2.574836982024013, 1)
(2.574836982024013, 1)
(0.4202676150502109, 1)
(2.341681953932289, 1)
(1.045630193236076, 1)
(2.341681953932289, 1)
(1.8306104706098087, 1)
(0.7535492018748609, 1)
(2.02253412315468, 1)
(2.574836982024013, 1)
(0.7535492018748609, 1)
(0.4142809311761813, 1)
(0.4518638216531902, 1)
(2.574836982024013, 1)
(0.4202676150502109, 1)
(1.1200432714506303, 1)
(0.7535492018748609, 1)
(0.5181550542899774, 1)
(2.341681953932289, 1)
(1.8626419592629413, 1)
(0.31961585322723524, 1)
(1.1200432714506303, 1)
(2.341681953932289, 1)
(2.02253412315468, 1)
(0.5181550542899774, 1)
(0.7535492018748609, 1)
(2.341681953932289, 1)
(1.8306

In [13]:
with open('sensors', 'w') as f:
    for index, sensor in enumerate(sensors):
        #print(index, sensor)
        f.write('{} {} {} {}\n'.format(sensor[0], sensor[1], std_costs[index][0], std_costs[index][1]))

# Generate Covariance

In [14]:
with open('cov', 'w') as f:
    grid_len = 1000
    cov = np.zeros((grid_len, grid_len))
    for i in range(grid_len):
        for j in range(grid_len):
            if i == j:
                cov[i, j] = std_costs[i][0] ** 2
            f.write('{} '.format(cov[i, j]))
        f.write('\n')

# Generate Hypothesis

#### First, remove the outliers (pathloss < 80)

In [55]:
hypo_template = 'tx_{}_pathloss.txt'
hypo = None
for i in range(1, 4097):
    filename = hypo_template.format(i)
    hypo = pd.read_csv(filename, delimiter=' ', header=None)
    flag = False
    for j in range(64):
        for k in range(64):
            if hypo.iloc[j, k] < 80:
                flag = True
                print(i, j, hypo.iloc[j, k], end=' --> ')
                hypo.iat[j, k] = 109 + np.random.rand()*2
                print(hypo.iloc[j, k])
    if flag:
        hypo.to_csv(filename, sep=' ', index=False, header=False)

16 21 75.4198 --> 109.11904254310691
61 52 70.8709 --> 110.9905276886906
78 2 72.534 --> 109.31234779210277
116 41 78.3035 --> 109.65423696149735
130 22 32.1242 --> 109.68767442818336
145 26 77.8135 --> 110.7472477638779
150 29 58.2435 --> 110.3535755696043
157 55 43.5814 --> 109.88569684419369
169 32 53.9384 --> 110.03240974732347
194 32 79.3567 --> 110.6822367047933
195 52 76.3047 --> 110.8818040574615
201 35 46.2602 --> 110.8779595124227
216 36 76.491 --> 109.73994290287376
222 59 77.6742 --> 109.12103594070776
239 52 78.1916 --> 110.12498612183374
269 26 19.0278 --> 109.79225288298933
287 33 65.0907 --> 110.2710268113381
309 59 79.7816 --> 110.89887732762142
313 50 58.6904 --> 110.19988404726001
330 29 66.6688 --> 110.80902374054534
333 30 78.8533 --> 110.81402401598162
385 1 53.3867 --> 110.40781864913039
386 11 78.9095 --> 110.12382476466016
398 10 69.8849 --> 110.73449035774826
400 13 66.548 --> 109.53386970679216
413 44 79.4481 --> 109.76927080590012
421 33 55.9509 --> 109.9299

#### Create hypothesis file

In [50]:
sensors = []
std_costs = []
with open('sensors', 'r') as f:
    lines = f.readlines()
    for line in lines:
        line = line.split()
        sensors.append((int(line[0]), int(line[1])))
        std_costs.append((float(line[2]), int(line[3])))

In [56]:
grid_len = 64
hypo_template = 'tx_{}_pathloss.txt'
hypo = None
hypothesis_file = open('hypothesis' ,'w')
for i in range(1, 4097):
    if i%100 == 0:
        print(i, end=' ')
    hypo = pd.read_csv(hypo_template.format(i), delimiter=' ', header=None)
    trans_x = (i-1)//grid_len
    trans_y = (i-1)%grid_len
    for index, sensor in enumerate(sensors):
        sen_x = sensor[0]
        sen_y = sensor[1]
        std = std_costs[index][0]
        mean = 30 - hypo.iloc[sen_y, sen_x]  # cellular tower power - pathloss
        print(trans_x, trans_y, sen_x, sen_y, mean, std, file=hypothesis_file)
hypothesis_file.close()

100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 3400 3500 3600 3700 3800 3900 4000 